In [2]:
import pandas as pd
df = pd.read_csv('train.csv')
df.head()

,answers,query,finalpassage
0,"Start around $5,000.",average kitchen cabinet remodel cost,"1 In contrast, DoItYourself.com estimates that..."
1,Jurassic World Evolution will launch during th...,when is jurassic world evolution coming out,JURASSIC WORLD EVOLUTION Trailer #2 (2018) Jur...
2,A fingerprint recognition feature.,what iphone introduced touch id,"With an iPhone 6 or iPhone 6 Plus or later, yo..."
3,foods contained hemoglobin,where does iron come from from food?,There are two forms of dietary iron: heme and ...
4,"Yes, thinning vocal cords cause hoarseness.",can thinning vocal cords cause hoarseness,"When you talk or sing, the vocal cords vibrate..."


In [3]:
df['finalpassage'][0]

'1 In contrast, DoItYourself.com estimates that the cost of a typical kitchen remodel can start around $5,000 with some do-it-yourself work included, and averages $15,000-$26,000. 1 These are averages; actual costs may be higher or lower. 2 The cost of remodeling a kitchen can vary widely depending on the size and scope of your project, with the national average coming in around $17,000 and most homeowners spending between $12,000 and $19,000. This guide can help estimate your costs and give you a starting point for various upgrades and services. The average cost of a complete kitchen remodel will run anywhere from a low of $7,504 all the way to the maximum of $40,000.'

In [4]:
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv('train.csv')

train, temp = train_test_split(df, test_size=0.2, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)

train.to_csv('train.csv', index=False)
val.to_csv('val.csv', index=False)
test.to_csv('test.csv', index=False)


In [11]:
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

import pandas as pd
from datasets import Dataset

train_df = pd.read_csv('train.csv')
val_df = pd.read_csv('val.csv')

train_df = train_df.dropna(subset=['finalpassage','query'])
val_df = val_df.dropna(subset=['finalpassage','query'])

train_df = train_df.sample(1000, random_state=42)
val_df = val_df.sample(200, random_state=42)

def preprocess(df):
    return Dataset.from_pandas(df[['finalpassage','query']])

train_dataset = preprocess(train_df)
val_dataset = preprocess(val_df)

tokenizer = T5Tokenizer.from_pretrained('t5-small')

def tokenize(batch):
    passages = [p if p is not None else "" for p in batch['finalpassage']]
    questions = [q if q is not None else "" for q in batch['query']]

    source = ["generate question: " + p for p in passages]
    target = questions

    model_inputs = tokenizer(source, max_length=512, truncation=True, padding='max_length')
    labels = tokenizer(target, max_length=64, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)




Map: 100%|██████████| 200/200 [00:00<00:00, 991.03 examples/s] 


In [13]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')

training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=3e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir='./logs',
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

trainer.save_model("./final_model_test")
tokenizer.save_pretrained("./final_model_test")


c:\Users\nikhi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
500,0.455700


('./final_model_test\\tokenizer_config.json',
 './final_model_test\\special_tokens_map.json',
 './final_model_test\\spiece.model',
 './final_model_test\\added_tokens.json')

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_path = "./final_model_test"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

qg_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer
)

passage = "The sun is the star at the center of the Solar System. It provides light and heat to the planets."

result = qg_pipeline(
    "generate question: " + passage,
    max_new_tokens=64,
    do_sample=False
)

print(result[0]['generated_text'])


Device set to use cpu


what is the sun at the center of the solar system
